**TESTING A LLM IN SAGEMAKER** 

+ GO TO [HugginFace model](https://huggingface.co/MBZUAI/LaMini-T5-738M)
+ Deploy/Sagemaker

In [ ]:
##install some libraries
!pip install transformers einops accelerate bitsandbytes

In [ ]:
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64


In [ ]:
hf_model='MBZUAI/LaMini-T5-738M' ##model in huggingFace to use

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(hf_model)
base_model= AutoModelForSeq2SeqLM.from_pretrained(hf_model,device_map='auto',torch_dtype=torch.float32)

In [ ]:
!pip install langchain #to import hugginF.

In [ ]:
from langchain.llms import HuggingFacePipeline

In [ ]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample=True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [ ]:
in_prompt='Write a shortly article on Rockets and IA'

In [ ]:
model=llm_pipeline()
gen_text=model(in_prompt)
print(f'input={in_prompt}\nresponse:{gen_text}')

In [ ]:
!pip uninstall -y sagemaker ##by the version

In [ ]:
!pip install sagemaker==2.168.0 ##version what we need

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models

hub = {
	'HF_MODEL_ID':hf_model,
	'HF_TASK': ''
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "how can I become more healthy?",
})